<a href="https://colab.research.google.com/github/guilhermecarva/guilhermecarva/blob/main/Deteccao_Anomalias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>

## __Exercício: Detecção de Anomalias__

<br>

__1:__

Utilizando a classe DetectorAnomalias criada ao longo do módulo, __vamos avaliar um detector de anomalias.__

O dataset utilizado pode ser importado através da função getData.

Nesse conjunto de dados, possuímos 6 variáveis explicativas, $X_1, .., X_6$ e uma variável com a marcação se a instância é uma anomalia ou não.

Utilizando a __metodolodia__ discutida ao longo do módulo, __teste diferentes modelos (variando o limiar $\epsilon$)__ a fim de encontrar o que __melhor fita os dados.__

Justifique as escolhas do $\epsilon$, bem como quais as métricas de performance abordadas.

<br>

__2:__

Aborde o problema num contexto de aprendizado supervisionado, ou seja, treine modelos de classificação binária com o objetivo de detectar anomalias.

Compare os resultados entre as metodologias.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

In [ ]:
class DetectorAnomalias():

    def __init__(self, epsilon):
        self.epsilon = epsilon

    def fit(self, X):
        medias = X.mean(axis = 0)
        desvios = X.std(axis = 0)
        gaussianas = [st.norm(loc = m, scale = d) for m, d in zip(medias, desvios)]
        self.gaussianas = gaussianas
        self.X = X

    def prob(self, x):
        p = 1
        for i in range(self.X.shape[1]):
            gaussiana_i = self.gaussianas[i]
            x_i = x[i]
            p *= gaussiana_i.pdf(x_i)
        return p

    def isAnomaly(self, x):
        return int(np.where(self.prob(x) < self.epsilon, 1, 0))

In [ ]:
def getData():
    return pd.read_csv("dataframe_anomalias_exercicio.csv")

In [ ]:
df = getData()
df

,x1,x2,x3,x4,x5,x6,anomalia
0,7.731153,23.299155,-0.367453,4.715372,9.306179,16.780965,0.0
1,11.466833,16.943695,-0.245131,7.060311,10.462826,19.821289,0.0
2,11.501272,20.196011,1.206049,-4.957189,7.771262,19.100079,0.0
3,10.893921,16.072385,2.738045,-3.684228,7.373334,23.225524,0.0
4,10.091706,19.253894,0.996895,-9.504052,8.883988,17.903298,0.0
...,...,...,...,...,...,...,...
10095,11.192286,18.451987,-0.953650,-14.362996,10.875826,17.056541,0.0
10096,12.014177,19.461815,1.985099,-7.119190,11.079922,17.582755,0.0
10097,10.745460,18.175951,0.206037,-1.897015,9.888329,17.963324,0.0
10098,9.893969,22.333270,-1.465981,4.137382,7.690620,21.570097,0.0


In [ ]:
df.anomalia.value_counts()

0.0    10046
1.0       54
Name: anomalia, dtype: int64

### Vamos primeiramente criar um dataset sem a coluna anomalia para ser utilizado no detector de anomalias com diferentes limiares

In [ ]:
# criando um df para X  (variáveis explicativas)
df_x = df.iloc[:,:6]
df_x.shape

(10100, 6)

In [ ]:
# criando um df para y (variável com marcação)
df_y = df.anomalia
df_y.shape

(10100,)

In [ ]:
df_x.head()

,x1,x2,x3,x4,x5,x6
0,7.731153,23.299155,-0.367453,4.715372,9.306179,16.780965
1,11.466833,16.943695,-0.245131,7.060311,10.462826,19.821289
2,11.501272,20.196011,1.206049,-4.957189,7.771262,19.100079
3,10.893921,16.072385,2.738045,-3.684228,7.373334,23.225524
4,10.091706,19.253894,0.996895,-9.504052,8.883988,17.903298


In [ ]:
df_y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: anomalia, dtype: float64

### Vamos utilizar a classe detector de anomalias

In [ ]:
# definindo valores para X
X = df_x.values

In [ ]:
# definindo valores para y
y = df_y.values

In [ ]:
#instanciando o algoritmo
ann = DetectorAnomalias(epsilon = 0.001)
ann.fit(X)

In [ ]:
#fazendo testes
print('marcação: ', y[0], 'probabilidade: ', ann.prob(X[0]), 'classificação: ', ann.isAnomaly(X[0]))

marcação:  0.0 probabilidade:  4.1448586959619875e-07 classificação:  1


### Vamos agora utilizar difrentes limiares para descobrir qual obtem a melhor nota com a métrica AUC ( já que temos uma classificação binária e é uma métrica interessante para tarefas com classes desproporcionais )

In [ ]:
from sklearn import metrics

In [ ]:
auc_score=[]
ep = [10**-2, 10**-3, 10**-4, 10**-5, 10**-6, 10**-7, 10**-8, 10**-9]
for eps in ep:
    ann = DetectorAnomalias(epsilon = eps)
    ann.fit(X)
    pred=[]
    for i in range(len(X)):
        pred.append(ann.isAnomaly(X[i,]))
    fpr, tpr, thresholds = metrics.roc_curve(y, pred)
    auc_score.append(metrics.auc(fpr, tpr))
    print('O score utilizando o epsilon (',eps,') foi de :', metrics.auc(fpr, tpr))

O score utilizando o epsilon ( 0.01 ) foi de : 0.5
O score utilizando o epsilon ( 0.001 ) foi de : 0.5
O score utilizando o epsilon ( 0.0001 ) foi de : 0.5
O score utilizando o epsilon ( 1e-05 ) foi de : 0.6675293649213617
O score utilizando o epsilon ( 1e-06 ) foi de : 0.908222177981286
O score utilizando o epsilon ( 1e-07 ) foi de : 0.9837248656181564
O score utilizando o epsilon ( 1e-08 ) foi de : 0.9996018315747561
O score utilizando o epsilon ( 1e-09 ) foi de : 0.5925925925925926


### Dentre os limiares testados, o epsilon com maior score foi o 10**-8. então vamos utilizá-lo para treinar o modelo / detector de anomalia. E aplicar algumas métricas de classificação binária nos datasets de validação e teste.
### Vamos começar dividindo no dataset em treino , teste e validação.

In [ ]:
#dataframe sem marcação
df_nan = df[df.anomalia==0]
df_nan.shape

(10046, 7)

In [ ]:
#dataframe somente da marcação
df_an = df[df.anomalia==1]
df_an.shape

(54, 7)

In [ ]:
treino = df_nan[:6046] #vamos utilizar 6000 linhas sem marcação para o treino
valida = pd.concat([df_nan[6046:8046],df_an[:27]]) #2000 linhas com marcação para validação
teste = pd.concat([df_nan[8046:],df_an[27:]]) #2000 linhas com marcação para teste
treino.shape, valida.shape, teste.shape

((6046, 7), (2027, 7), (2027, 7))

In [ ]:
x_treino = treino.drop('anomalia', axis=1).values
y_treino = treino.anomalia.values

x_valida = valida.drop('anomalia', axis=1).values
y_valida = valida.anomalia.values

x_teste = teste.drop('anomalia', axis=1).values
y_teste = teste.anomalia.values

### Tendo os datasets já separados vamos ver se obtemos um bom score nos dataset de validação e teste

In [ ]:
ann = DetectorAnomalias(epsilon=10**-8)
ann.fit(x_treino)
y_pred=[]
for i in range(len(y_valida)):
    y_pred.append(ann.isAnomaly(x_valida[i, ]))

fpr, tpr, thresholds = metrics.roc_curve(y_valida, y_pred)
print('AUC Score no dataset de validação ',round(metrics.auc(fpr, tpr),5)*100,'%')

AUC Score no dataset de validação  99.95 %


In [ ]:
ann = DetectorAnomalias(epsilon=10**-8)
ann.fit(x_treino)
y_pred=[]
for i in range(len(y_teste)):
    y_pred.append(ann.isAnomaly(x_teste[i, ]))

fpr, tpr, thresholds = metrics.roc_curve(y_teste, y_pred)
print('AUC Score no dataset de teste ',round(metrics.auc(fpr, tpr),5)*100,'%')

AUC Score no dataset de teste  99.9 %


### Em ambos datasets obtivemos um score bem alto <br> Vamos agora testar outras métricas

In [ ]:
from sklearn.metrics import accuracy_score #número de previsões corretas pelo número total de previsões
from sklearn.metrics import f1_score #média harmônica entre a precisão e o recall
from sklearn.metrics import precision_score #mede o quanto podemos confiar num modelo , mede a quantidade de vezes que o seu modelo acerta em relação ao total de vezes que ele tenta acertar
from sklearn.metrics import recall_score #quantidade de vezes que o seu modelo acerta em relação ao total de vezes que ele deveria ter acertado

In [ ]:
print('Accuracy Score no dataset de teste ',round(accuracy_score(y_teste, y_pred),5)*100,'%')
print('F1 Score no dataset de teste ',round(f1_score(y_teste, y_pred),2)*100,'%')
print('Precision Score no dataset de teste ',round(precision_score(y_teste, y_pred),2)*100,'%')
print('Recall Score no dataset de teste ',round(recall_score(y_teste, y_pred),2)*100,'%')

Accuracy Score no dataset de teste  99.803 %
F1 Score no dataset de teste  93.0 %
Precision Score no dataset de teste  87.0 %
Recall Score no dataset de teste  100.0 %


### Com uma acurácia de 99,8%. A precisão embora seja alta, não acompanha o score dos demais talvez or não termos classes muito balanceadas, ou seja, a representatividade de amostras anômalas é muito pequena em relação as normais.